<a href="https://colab.research.google.com/github/microprediction/timeseries-notebooks/blob/main/simdkalman_hello_again.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade numpy
!pip install --upgrade git+https://github.com/oseiskar/simdkalman



# Update
Testing updating of different systems simultaneously


In [1]:
from simdkalman import KalmanFilter
import numpy as np 

# Process noise 
Q = np.zeros((2,3,3))
Q[0,:,:] = np.eye(3)
Q[1,:,:] = np.eye(3)


# Prior state means 
m0 = np.zeros((2,3,1))
m0[0,:,:] = np.array([[1.5, 1.5, 1.5]]).transpose()
m0[1,:,:] = np.array([[1.5, 1.5, 1.5]]).transpose()

# Prior covariances
P0 = np.zeros((2, 3, 3))
P0[0, :, :] = np.eye(3)
P0[1, :, :] = 0.5 * np.eye(3)

# Observations
y_update = np.zeros((2, 1, 1))
y_update[0, :, :] = np.array([[-1.4]])
y_update[1, :, :] = np.array([[-0.33]])

# Measurement var
R = np.zeros((2, 1, 1))
R[0, :, :] = np.eye(1)
R[1, :, :] = 0.66*np.eye(1)

# Observation equations 
H0 = np.array([[0.8, 0.2, 0]])
H1 = np.array([[0.9, 0.1, 0]])
H0.shape = (1, 3)
H1.shape = (1, 3)
H = np.zeros((2, 1, 3))
H[0, :, :] = H0
H[1, :, :] = H1

# Transition(s) 
testing_different_transitions = False #<-- This can be set to True after my PR is accepted
A0 = np.array([[0.5, 0.3, 0.2], [1, 0, 0], [0, 1, 0]])
A = A0 
kf = KalmanFilter(
    state_transition=A,   # A
    process_noise=Q,      # Q
    observation_model=H,  # H
    observation_noise=R)  # R
m1, P1, K, ll = kf.update(m0,P0,y_update, log_likelihood=True)
print(m1)



[[[0.11904762]
  [1.1547619 ]
  [1.5       ]]

 [[0.73037383]
  [1.41448598]
  [1.5       ]]]


# Prediction (evolution)

In [3]:
results = kf.predict(data=y_update,n_test=1,initial_value=m0, initial_covariance=P0,
                         states=True, covariances=True)
m1 = results.states.mean
P1 = results.states.cov
print(m1)

[[[0.70595238 0.11904762 1.1547619 ]]

 [[1.08953271 0.73037383 1.41448598]]]


### Now again, but this time with different transitions
This will only work in a recent version

In [6]:
A = np.zeros((2,3,3) )
A1 = np.array([[0.4, 0.4, 0.2], [1, 0, 0], [0, 1, 0]])
A[0,:,:] = A0
A[1,:,:] = A1
kf = KalmanFilter(
    state_transition=A,   # A
    process_noise=Q,      # Q
    observation_model=H,  # H
    observation_noise=R)  # R (previous version of code may throw error)
results = kf.predict(data=y_update,n_test=1,initial_value=m0, initial_covariance=P0,
                         states=True, covariances=True)
m1 = results.states.mean
P1 = results.states.cov
print(m1)

AssertionError: ignored